In [ ]:
import os

In [1]:
%pwd

'c:\\Users\\welcome\\Desktop\\End-to-End-mlproject-in-MLops\\researches'

In [2]:
import os
os.chdir("../")

In [3]:
%pwd

'c:\\Users\\welcome\\Desktop\\End-to-End-mlproject-in-MLops'

In [12]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [24]:
from pathlib import Path
from src.utils import read_yamls, create_directories

CONFIG_FILE_PATH=Path("yamls/config.yaml")


class ConfigurationManager:
    def __init__(
        self,
        config_file_path = CONFIG_FILE_PATH):
      

        self.config = read_yamls(config_file_path)
       

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_injection

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_url,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_path 
        )

        return data_ingestion_config

In [25]:
import os
import urllib.request as request
import zipfile
from src.logger import logging
from src.utils import get_size
from src.exceptions import CustomException
import sys
from pathlib import Path

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logging.info(f"{filename} download! with following info: \n{headers}")
        else:
            logging.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")



    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
  

In [26]:
%pwd

'c:\\Users\\welcome\\Desktop\\End-to-End-mlproject-in-MLops'

In [27]:
import sys
from src.exceptions import CustomException

try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise CustomException(e,sys)